In [1]:
#imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img, img_to_array
import numpy as np

from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.applications.resnet50 import ResNet50

In [4]:
class Metrics(keras.callbacks.Callback):
    """
    Implementation of custom metrics: Precision, Recall, F-Measure and Confusion Matrix
    """
    
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        
        for i in range(len(self.validation_data)):
            x_val, y_val = self.validation_data.__getitem__(i)

        print(y_val.shape)
        print(y_val[1])
        print(y_val[2])

        y_predict = np.asarray(model.predict(self.validation_data, steps = 1))
        print(y_predict.shape)

        with tf.Session() as sess:
            lab = [i for i in range(0,75)]
            print(y_val.shape)

            y_val = np.argmax(y_val, axis=1)
            y_predict = np.argmax(y_predict, axis=1)
            print("y_val: ", y_val)
            print("y_predict:", y_predict)
            print("\nMetrics for Epoch")
            print("Confusion Matrix:\n",confusion_matrix(y_val,y_predict, labels=lab))
            print("Recall: ", recall_score(y_val,y_predict, average=None, labels=lab))
            print("Precision: ", precision_score(y_val,y_predict, average = None, labels=lab))
            print("F1_score: ", f1_score(y_val,y_predict, average =None, labels=lab))
            print("\n") 
            self._data.append({
                'val_recall': recall_score(y_val, y_predict, average = None, labels=lab),
                'val_precision': precision_score(y_val, y_predict, average = None, labels=lab),
                'val_f1_score': f1_score(y_val,y_predict, average = None, labels=lab),
            })
            return

    def get_data(self):
        return self._data
    
metrics = Metrics()

In [5]:
def lr_scheduler(epoch,lr):
    
    """
    Learning rate scheduler decays the learning rate by factor of 0.1 every 10 epochs after 20 epochs
    """
    decay_rate = 0.1
    if epoch==20:
        return lr*decay_rate
    elif epoch%10==0 and epoch >20:
        return lr*decay_rate
    return lr

# To put in the model, put it inside callbacks
LRScheduler = keras.callbacks.LearningRateScheduler(lr_scheduler,verbose=1)

In [2]:
#Data Generator for Data augmentation. Edit possible
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split = 0.008)

In [3]:
#Sanity check for augmentation

img = load_img('../datasets/large_sample/almond_desert/6350.jpg')  # this is a PIL image
img=img.resize((100,100))
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
print(x.shape)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in train_datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='almond_desert', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

(100, 100, 3)


In [16]:
train_generator = train_datagen.flow_from_directory(
        '../datasets/large_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical',
        subset = 'training')  # since we use binary_crossentropy loss, we need binary labels



Found 45509 images belonging to 75 classes.


In [17]:
val_generator = train_datagen.flow_from_directory(
        '../datasets/large_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=333,
        class_mode='categorical',
        subset = 'validation')  # since we use binary_crossentropy loss, we need binary labels

Found 333 images belonging to 75 classes.


In [8]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

{0: 'almond_desert', 1: 'apple_pine', 2: 'baby_cabbage', 3: 'breakfast_burritos', 4: 'cake', 5: 'candied_haw', 6: 'chinese_mitten_crab', 7: 'clam_juice_chicken_feet', 8: 'coconut_cake', 9: 'coffee', 10: 'cool_noodles', 11: 'corn_cake', 12: 'donuts', 13: 'double_cooked_pork_slices', 14: 'dry_fried_noodle', 15: 'duck_neck', 16: 'dumpling_ham', 17: 'dumplings', 18: 'egg_pudding', 19: 'egg_tart', 20: 'egg_waffle', 21: 'egg_yolk_puff', 22: 'ferment_cake', 23: 'french_toast', 24: 'fried_dough_twists', 25: 'fried_rice', 26: 'fruit_salad', 27: 'grilled_cold_noodles', 28: 'grilled_fish', 29: 'gui_ling_jelly', 30: 'hong_kong_style_french_toast', 31: 'hot_and_sour_soup', 32: 'hot_pot', 33: 'huntun', 34: 'ice_cream', 35: 'mango_crepe', 36: 'mashed_potato', 37: 'milk_pudding', 38: 'millet_porridge', 39: 'mixed_porridge', 40: 'moon_cake', 41: 'pancake', 42: 'pickle', 43: 'pineapple_buns', 44: 'pizza', 45: 'pork_knuckle', 46: 'potato_shred', 47: 'puffs', 48: 'pumpkin_porridge', 49: 'red_bean_pudding'

In [9]:
#Naive Model
input_tensor = keras.layers.Input(shape = (28,28,3))
model = ResNet50(input_tensor=input_tensor,include_top=False, classes = 75, weights = None)
x=model.output

x= tf.keras.layers.GlobalAveragePooling2D(data_format='channels_last')(x)
x=tf.keras.layers.Dense(75,activation ='softmax')(x)
model = tf.keras.models.Model(model.input, x)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


/home/mudit/anaconda3/envs/TensorflowGPU/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [10]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 34, 34, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 14, 14, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 14, 14, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [11]:
model.fit_generator(train_generator,
                   steps_per_epoch = 1,
                   epochs = 1, validation_data = val_generator)

Epoch 1/1
1/1 [==============================] - 17s 17s/step - loss: 6.5780 - acc: 0.0000e+00 - val_loss: 16.1181 - val_acc: 0.0000e+00


In [12]:
img = load_img('../datasets/large_sample/almond_desert/6350.jpg')  # this is a PIL image
img=img.resize((28,28))
display(img)
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

print(x.shape)

(1, 28, 28, 3)


In [13]:

y = model.predict(val_generator, steps = 1)

In [14]:
labels[np.argmax(y[1])]

'double_cooked_pork_slices'

In [15]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation
5 pool1_pad
6 max_pooling2d
7 res2a_branch2a
8 bn2a_branch2a
9 activation_1
10 res2a_branch2b
11 bn2a_branch2b
12 activation_2
13 res2a_branch2c
14 res2a_branch1
15 bn2a_branch2c
16 bn2a_branch1
17 add
18 activation_3
19 res2b_branch2a
20 bn2b_branch2a
21 activation_4
22 res2b_branch2b
23 bn2b_branch2b
24 activation_5
25 res2b_branch2c
26 bn2b_branch2c
27 add_1
28 activation_6
29 res2c_branch2a
30 bn2c_branch2a
31 activation_7
32 res2c_branch2b
33 bn2c_branch2b
34 activation_8
35 res2c_branch2c
36 bn2c_branch2c
37 add_2
38 activation_9
39 res3a_branch2a
40 bn3a_branch2a
41 activation_10
42 res3a_branch2b
43 bn3a_branch2b
44 activation_11
45 res3a_branch2c
46 res3a_branch1
47 bn3a_branch2c
48 bn3a_branch1
49 add_3
50 activation_12
51 res3b_branch2a
52 bn3b_branch2a
53 activation_13
54 res3b_branch2b
55 bn3b_branch2b
56 activation_14
57 res3b_branch2c
58 bn3b_branch2c
59 add_4
60 activation_15
61 res3c_branch2a
62 bn3c_branch2a
63 act

In [16]:
#Data Generator for Data augmentation. Edit possible
val_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split = 0.3)

In [17]:
small_train_generator = val_datagen.flow_from_directory(
        '../datasets/small_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical',
        subset = 'training')

Found 100 images belonging to 25 classes.


In [18]:
small_val_generator = val_datagen.flow_from_directory(
        '../datasets/small_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical',
        subset = 'validation')

Found 25 images belonging to 25 classes.


In [19]:
for layer in model.layers[:-34]:
    layer.trainable=False


In [20]:
for i,layer in enumerate(model.layers):
  print(i,layer.trainable)

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 False
22 False
23 False
24 False
25 False
26 False
27 False
28 False
29 False
30 False
31 False
32 False
33 False
34 False
35 False
36 False
37 False
38 False
39 False
40 False
41 False
42 False
43 False
44 False
45 False
46 False
47 False
48 False
49 False
50 False
51 False
52 False
53 False
54 False
55 False
56 False
57 False
58 False
59 False
60 False
61 False
62 False
63 False
64 False
65 False
66 False
67 False
68 False
69 False
70 False
71 False
72 False
73 False
74 False
75 False
76 False
77 False
78 False
79 False
80 False
81 False
82 False
83 False
84 False
85 False
86 False
87 False
88 False
89 False
90 False
91 False
92 False
93 False
94 False
95 False
96 False
97 False
98 False
99 False
100 False
101 False
102 False
103 False
104 False
105 False
106 False
107 False
108 False
109 False
110 False


In [46]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
new_model = tf.keras.models.Model(model.input, model.layers[-3].output)

In [47]:
new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 34, 34, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 14, 14, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 14, 14, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [48]:
y=new_model.output

y= tf.keras.layers.GlobalAveragePooling2D(data_format='channels_last')(y)
y=tf.keras.layers.Dense(25,activation ='softmax')(y)

new_model = tf.keras.models.Model(new_model.input, y)

new_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [49]:
new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 34, 34, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 14, 14, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 14, 14, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [50]:
for i,layer in enumerate(model.layers):
  print(i,layer.trainable)

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 False
22 False
23 False
24 False
25 False
26 False
27 False
28 False
29 False
30 False
31 False
32 False
33 False
34 False
35 False
36 False
37 False
38 False
39 False
40 False
41 False
42 False
43 False
44 False
45 False
46 False
47 False
48 False
49 False
50 False
51 False
52 False
53 False
54 False
55 False
56 False
57 False
58 False
59 False
60 False
61 False
62 False
63 False
64 False
65 False
66 False
67 False
68 False
69 False
70 False
71 False
72 False
73 False
74 False
75 False
76 False
77 False
78 False
79 False
80 False
81 False
82 False
83 False
84 False
85 False
86 False
87 False
88 False
89 False
90 False
91 False
92 False
93 False
94 False
95 False
96 False
97 False
98 False
99 False
100 False
101 False
102 False
103 False
104 False
105 False
106 False
107 False
108 False
109 False
110 False


In [52]:
new_model.fit_generator(small_train_generator,
                   steps_per_epoch = 100,
                   epochs = 1, validation_data = small_val_generator)

Epoch 1/1
100/100 [==============================] - 39s 388ms/step - loss: 5.5476 - acc: 0.0528 - val_loss: 15.4734 - val_acc: 0.0400


In [53]:
img = load_img('../datasets/small_sample/black_rice_cake/10585.jpg')  # this is a PIL image
img=img.resize((28,28))
display(img)
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

print(x.shape)

(1, 28, 28, 3)


In [54]:
labelsSmall = (small_train_generator.class_indices)
labelsSmall = dict((v,k) for k,v in labelsSmall.items())
print(labelsSmall)

{0: 'Omelette', 1: 'Snowflake_Flaky_Pastry', 2: 'black_rice_cake', 3: 'black_sesame_paste', 4: 'boiled_corn', 5: 'braised_pork', 6: 'cheese', 7: 'claypot', 8: 'french_fries', 9: 'fried_chestnuts', 10: 'frozen_lemon_tea', 11: 'glutinous_rice_ball_with_peanut_and_sesame_sweets', 12: 'glutinous_rice_dumpling', 13: 'green_bean_cake', 14: 'hot_dog', 15: 'layer_cake', 16: 'mango_pomelo_sago', 17: 'onion_rings', 18: 'persimmon_pancake', 19: 'poon_choi', 20: 'pop_corn', 21: 'sampan_porridge', 22: 'scrambled_eggs_with_tomatoes', 23: 'steak', 24: 'stir_fried_lettuce'}


In [55]:
y = model.predict(x)

In [56]:
labels[np.argmax(y)]

'sushi'